In [170]:
library(dplyr)

In [174]:
proj_tcga_sel <- c("tcga_brca", "tcga_coad","tcga_esca","tcga_lgg","tcga_luad","tcga_lusc","tcga_paad","tcga_skcm","tcga_stad")
lapply(proj_tcga_sel, FUN=function(x) dir.create(paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/04_mm_ko_drug_used/04_1_correc_drug/",x, sep="")))

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

In [175]:
drug_bank <- read.table("/mnt/lustre/scratch/CBRA/data/DRUGBANK/drugbank_drug-bindings_v5.1.8.tsv", sep="\t", header=T)
drug_bank_sel <- unique(drug_bank[,c("drug_name","entrez_id")])
drugs_db <- tolower(unique(drug_bank_sel$drug_name))
length(proj_tcga_sel)

[1] 9

In [178]:
for (i in 1:length(proj_tcga_sel)){

    file <- paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/02_selection_samples/02_1_ext_information_patients_tcga/",proj_tcga_sel[i],"/patients_geneexp_clind_drug_samptype_",proj_tcga_sel[i],".rds",sep="")
    
    pat_drug <- as.data.frame(readRDS(file) %>% filter(sample_type=="Primary Solid Tumor") %>% select(bcr_patient_barcode, drug_name))

    # x
    plus_drug <- pat_drug[grep("+",pat_drug$drug_name, fixed=TRUE),]
 
    if (length(plus_drug$drug_name) != 0){
        pat_drug <- pat_drug[-grep("+",pat_drug$drug_name, fixed=TRUE),]
        sep_plus_drug <- lapply(plus_drug$drug_name, FUN=function(x) gsub("^ ","",gsub("$ ","",unlist(strsplit(x,"+",fixed=T)))))
        pat_drug <- rbind(pat_drug,
        data.frame("bcr_patient_barcode"= unlist(
        lapply(1:length(sep_plus_drug), FUN=function(x)                       
        rep(plus_drug$bcr_patient_barcode[x], length(sep_plus_drug[[x]])))),
        "drug_name"=unlist(sep_plus_drug)))  }

    # /               
    slash_drug <- pat_drug[grep("/",pat_drug$drug_name, fixed=TRUE),]
    
    if (length(slash_drug$drug_name) != 0){
        pat_drug <- pat_drug[-grep("/",pat_drug$drug_name, fixed=TRUE),]
        sep_slash_drug <- lapply(slash_drug$drug_name, FUN=function(x) gsub("^ ","",gsub("$ ","",unlist(strsplit(x,"/",fixed=T)))))
        pat_drug <- rbind(pat_drug,
        data.frame("bcr_patient_barcode"= unlist(
        lapply(1:length(sep_slash_drug), FUN=function(x)                       
        rep(slash_drug$bcr_patient_barcode[x], length(sep_slash_drug[[x]])))),
        "drug_name"=unlist(sep_slash_drug)))}              

# and
    and_drug <- pat_drug[grep(" and ",pat_drug$drug_name, fixed=TRUE),]
    

    if (length(and_drug$drug_name) != 0){
        pat_drug <- pat_drug[-grep(" and ",pat_drug$drug_name, fixed=TRUE),]
        sep_and_drug <- lapply(and_drug$drug_name, FUN=function(x) gsub("^ ","",gsub("$ ","",unlist(strsplit(x,"and",fixed=T)))))
        pat_drug <- rbind(pat_drug,
        data.frame("bcr_patient_barcode"= unlist(
        lapply(1:length(sep_and_drug), FUN=function(x)                       
        rep(and_drug$bcr_patient_barcode[x], length(sep_and_drug[[x]])))),
        "drug_name"=unlist(sep_and_drug)))}
    # ,
    com_drug <- pat_drug[grep(",",pat_drug$drug_name, fixed=TRUE),]

    if (length(com_drug$drug_name) != 0){
        pat_drug <- pat_drug[-grep(",",pat_drug$drug_name, fixed=TRUE),]
        sep_com_drug <- lapply(com_drug$drug_name, FUN=function(x) gsub("^ ","",gsub("$ ","",unlist(strsplit(x,", ",fixed=T)))))
        pat_drug <- rbind(pat_drug,
        data.frame("bcr_patient_barcode"= unlist(
        lapply(1:length(sep_com_drug), FUN=function(x)                       
        rep(com_drug$bcr_patient_barcode[x], length(sep_com_drug[[x]])))),
        "drug_name"=unlist(sep_com_drug)))}
                  
    if (length(pat_drug[which(pat_drug$drug_name == "doxorubicine cyclophosphamide tamoxifen"),"bcr_patient_barcode"]) != 0){
        pat_drug <- rbind(pat_drug,
        data.frame("bcr_patient_barcode"= rep(pat_drug[which(pat_drug$drug_name == "doxorubicine cyclophosphamide tamoxifen"),"bcr_patient_barcode"], 3),
        "drug_name"=c("doxorubicine","cyclophosphamide","tamoxifen")))
        pat_drug <- pat_drug[-which(pat_drug$drug_name == "doxorubicine cyclophosphamide tamoxifen"),]}

    pat_drug$drug_name[which(pat_drug$drug_name == "pelareorep")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "laferon")] <- "interferon alfa-n3"                   
    pat_drug$drug_name[which(pat_drug$drug_name == "alferon")] <- "interferon alfa-n3"                   
    pat_drug$drug_name[which(pat_drug$drug_name == "zofran")] <- "ondansetron"                   
    pat_drug$drug_name[which(pat_drug$drug_name == "ciclosporin")] <- "cyclosporine"                   
    pat_drug$drug_name[which(pat_drug$drug_name == "renamin")] <- ""                   
    pat_drug$drug_name[which(pat_drug$drug_name == "ras inhibitor")] <- ""    
    pat_drug$drug_name[which(pat_drug$drug_name == "tarceva")] <- "erlotinib"        
    pat_drug$drug_name[which(pat_drug$drug_name == "gliadel wafer")] <- "carmustine"        
    pat_drug$drug_name[which(pat_drug$drug_name == "zoladex")] <- "goserelin"
    pat_drug$drug_name[which(pat_drug$drug_name == "poly e")] <- "colistin"
    pat_drug$drug_name[which(pat_drug$drug_name == "herceptin")] <- "trastuzumab"               
    pat_drug$drug_name[which(pat_drug$drug_name == "letrozol")] <- "letrozole"   
    pat_drug$drug_name[which(pat_drug$drug_name == "adriamycin")] <- "doxorubicin"
    pat_drug$drug_name[which(pat_drug$drug_name == "taxol")] <- "paclitaxel"                   
    pat_drug$drug_name[which(pat_drug$drug_name == "cytoxan")] <- "cyclophosphamide"
    pat_drug$drug_name[which(pat_drug$drug_name == "arimidex")] <- "anastrozole"
    pat_drug$drug_name[which(pat_drug$drug_name == "taxotere")] <- "docetaxel"
    pat_drug$drug_name[which(pat_drug$drug_name == "navelbine")] <- "vinorelbine"
    pat_drug$drug_name[which(pat_drug$drug_name == "xeloda")] <- "capecitabine"
    pat_drug$drug_name[which(pat_drug$drug_name == "gemzar")] <- "gemcitabine"
    pat_drug$drug_name[which(pat_drug$drug_name == "zometa")] <- "zoledronic acid"
    pat_drug$drug_name[which(pat_drug$drug_name == "e-75")] <- "equine botulinum neurotoxin e immune fab2"                   
    pat_drug$drug_name[which(pat_drug$drug_name == "capecetabine")] <- "capecitabine"
    pat_drug$drug_name[which(pat_drug$drug_name == "ae-37")] <- "blarcamesine"
    pat_drug$drug_name[which(pat_drug$drug_name == "anastrazole")] <- "anastrozole"
    pat_drug$drug_name[which(pat_drug$drug_name == "abraxane")] <- "paclitaxel"
    pat_drug$drug_name[which(pat_drug$drug_name == "doxorubicin liposome")] <- "doxorubicin"
    pat_drug$drug_name[which(pat_drug$drug_name == "lupron")] <- "leuprolide"
    pat_drug$drug_name[which(pat_drug$drug_name == "femara")] <- "letrozole"
    pat_drug$drug_name[which(pat_drug$drug_name == "aromasin")] <- "exemestane" 
    pat_drug$drug_name[which(pat_drug$drug_name == "avastin")] <- "bevacizumab"
    pat_drug$drug_name[which(pat_drug$drug_name == "adriamyicin")] <- "doxorubicin"
    pat_drug$drug_name[which(pat_drug$drug_name == "epirubicoin")] <- "epirubicin"
    pat_drug$drug_name[which(pat_drug$drug_name == "pamidronate")] <- "pamidronic acid"
    pat_drug$drug_name[which(pat_drug$drug_name == "aloxi")] <- "palonosetron"
    pat_drug$drug_name[which(pat_drug$drug_name == "5-fu")] <- "fluorouracil"  
    pat_drug$drug_name[which(pat_drug$drug_name == "faslodex")] <- "fulvestrant" 
    pat_drug$drug_name[which(pat_drug$drug_name == "xgeva")] <- "denosumab"
    pat_drug$drug_name[which(pat_drug$drug_name == "anastrozole (arimidex)")] <- "anastrozole"
    pat_drug$drug_name[which(pat_drug$drug_name == "exemestane (aromasin)")] <- "exemestane"
    pat_drug$drug_name[which(pat_drug$drug_name == "letrozole (femara)")] <- "letrozole"
    pat_drug$drug_name[which(pat_drug$drug_name == "tamoxifen (novadex)")] <- "tamoxifen"
    pat_drug$drug_name[which(pat_drug$drug_name == "flourouracil")] <- "fluorouracil"
    pat_drug$drug_name[which(pat_drug$drug_name == "cyclophasphamide")] <- "cyclophosphamide"
    pat_drug$drug_name[which(pat_drug$drug_name == "xeloda (capecitabine)")] <- "capecitabine"
    pat_drug$drug_name[which(pat_drug$drug_name == "femara (letrozole)")] <- "letrozole"
    pat_drug$drug_name[which(pat_drug$drug_name == "fulvestrant (faslodex)")] <- "fulvestrant" 
    pat_drug$drug_name[which(pat_drug$drug_name == "arimidex (anastrozole)")] <- "anastrozole"
    pat_drug$drug_name[which(pat_drug$drug_name == "aromasin (exemestane)")] <- "exemestane"
    pat_drug$drug_name[which(pat_drug$drug_name == "leuprolide acetate (lupron)")] <- "leuprolide"
    pat_drug$drug_name[which(pat_drug$drug_name == "cyclophospamide")] <- "cyclophosphamide"
    pat_drug$drug_name[which(pat_drug$drug_name == "albumin-bound paclitaxel")] <- "paclitaxel"
    pat_drug$drug_name[which(pat_drug$drug_name == "not otherwise specified")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "trustuzumab")] <- "trastuzumab"
    pat_drug$drug_name[which(pat_drug$drug_name == "mesna-1")] <- "coenzyme m"
    pat_drug$drug_name[which(pat_drug$drug_name == "mesna-2")] <- "coenzyme m"
    pat_drug$drug_name[which(pat_drug$drug_name == "vp-16")] <- "etoposide"
    pat_drug$drug_name[which(pat_drug$drug_name == "doxetaxel")] <- "doxycycline"
    pat_drug$drug_name[which(pat_drug$drug_name == "doxil")] <- "doxorubicin"
    pat_drug$drug_name[which(pat_drug$drug_name == "cyotxan")] <- "cyclophosphamide"
    pat_drug$drug_name[which(pat_drug$drug_name == "adrimycin")] <- "doxorubicin"
    pat_drug$drug_name[which(pat_drug$drug_name == "5 fluorouracil")] <- "fluorouracil"
    pat_drug$drug_name[which(pat_drug$drug_name == "doxorubicine")] <- "doxorubicin"
    pat_drug$drug_name[which(pat_drug$drug_name == "cyclophosphamid")] <- "cyclophosphamide"
    pat_drug$drug_name[which(pat_drug$drug_name == "metotreksat")] <- "methotrexate"
    pat_drug$drug_name[which(pat_drug$drug_name == "nolvadex")] <- "tamoxifen"
    pat_drug$drug_name[which(pat_drug$drug_name == "adriamicin")] <- "doxorubicin"
    pat_drug$drug_name[which(pat_drug$drug_name == "letrozolum")] <- "letrozole"
    pat_drug$drug_name[which(pat_drug$drug_name == "anastrozolum")] <- "anastrozole"
    pat_drug$drug_name[which(pat_drug$drug_name == "paclitaxelum")] <- "paclitaxel"
    pat_drug$drug_name[which(pat_drug$drug_name == "tamoxiphene")] <- "tamoxifen"
    pat_drug$drug_name[which(pat_drug$drug_name == "cyclophosphamidum")] <- "cyclophosphamide"
    pat_drug$drug_name[which(pat_drug$drug_name == "doxorubicinum")] <- "doxorubicin"
    pat_drug$drug_name[which(pat_drug$drug_name == "paclitaxel (protein-bound)")] <- "paclitaxel"
    pat_drug$drug_name[which(pat_drug$drug_name == "fareston")] <- "toremifene"    
    pat_drug$drug_name[which(pat_drug$drug_name == "taxane")] <- "vinorelbine"
    pat_drug$drug_name[which(pat_drug$drug_name == "clodronate")] <- "clodronic acid"
    pat_drug$drug_name[which(pat_drug$drug_name == "bevacizumab or placebo")] <- "bevacizumab"
    pat_drug$drug_name[which(pat_drug$drug_name == "cyclophosphane")] <- "cyclophosphamide"
    pat_drug$drug_name[which(pat_drug$drug_name == "leuprorelin")] <- "leuprolide"
    pat_drug$drug_name[which(pat_drug$drug_name == "aromatase exemestane")] <- "exemestane"
    pat_drug$drug_name[which(pat_drug$drug_name == "tch")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "cytoxen")] <- "cyclophosphamide"
    pat_drug$drug_name[which(pat_drug$drug_name == "megace")] <- "megestrol acetate"
    pat_drug$drug_name[which(pat_drug$drug_name == "yondelis")] <- "trabectedin"
    pat_drug$drug_name[which(pat_drug$drug_name == "tamoxifen citrate")] <- "tamoxifen"
    pat_drug$drug_name[which(pat_drug$drug_name == "doxorubicin hcl")] <- "doxorubicin"        
    pat_drug$drug_name[which(pat_drug$drug_name == "neulasta")] <- "pegfilgrastim"
    pat_drug$drug_name[which(pat_drug$drug_name == "adrimicin")] <- "doxorubicin"
    pat_drug$drug_name[which(pat_drug$drug_name == "leuporeline")] <- "leuprolide"
    pat_drug$drug_name[which(pat_drug$drug_name == "gosereline")] <- "goserelin"
    pat_drug$drug_name[which(pat_drug$drug_name == "methotrexatum")] <- "methotrexate"
    pat_drug$drug_name[which(pat_drug$drug_name == "fluorouracillum")] <- "fluorouracil"
    pat_drug$drug_name[which(pat_drug$drug_name == "tamoxiphen")] <- "tamoxifen"
    pat_drug$drug_name[which(pat_drug$drug_name == "anastrazolum")] <- "anastrozole"
	pat_drug$drug_name[which(pat_drug$drug_name == "cuclophosphamide")] <- "cyclophosphamide"                   
    pat_drug$drug_name[which(pat_drug$drug_name == "bevacizumab (avastin)")] <- "bevacizumab"
    pat_drug$drug_name[which(pat_drug$drug_name == "placebo provided by study")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "chemo")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "nos")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "hormone")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "tc")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "ac")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "cytoxan ")] <- "cyclophosphamide"
    pat_drug$drug_name[which(pat_drug$drug_name == "study drug amg 655")] <- "conatumumab"
    pat_drug$drug_name[which(pat_drug$drug_name == "5 fu")] <- "fluorouracil"
    pat_drug$drug_name[which(pat_drug$drug_name == "5fu")] <- "fluorouracil"
    pat_drug$drug_name[which(pat_drug$drug_name == "erbitux")] <- "cetuximab"    
    pat_drug$drug_name[which(pat_drug$drug_name == "levcovorin")] <- "levovirin"
    pat_drug$drug_name[which(pat_drug$drug_name == "5- fu")] <- "fluorouracil"
    pat_drug$drug_name[which(pat_drug$drug_name == "mitomycin c")] <- "mitomycin"
    pat_drug$drug_name[which(pat_drug$drug_name == "cetuximab study drug")] <- "cetuximab"
    pat_drug$drug_name[which(pat_drug$drug_name == "folinic acid")] <- "leucovorin"    
    pat_drug$drug_name[which(pat_drug$drug_name == "5-fluoruoracil")] <- "fluorouracil"           
    pat_drug$drug_name[which(pat_drug$drug_name == "folfox")] <- "fluorouracil"
    pat_drug$drug_name[which(pat_drug$drug_name == "camptosar")] <- "irinotecan"
    pat_drug$drug_name[which(pat_drug$drug_name == "cpt-11")] <- "irinotecan" 
    pat_drug$drug_name[which(pat_drug$drug_name == "folfiri")] <- "fluorouracil"                        
    pat_drug$drug_name[which(pat_drug$drug_name == "irinotecan hcl")] <- "irinotecan"
    pat_drug$drug_name[which(pat_drug$drug_name == "leucovorin calcium")] <- "leucovorin"
    pat_drug$drug_name[which(pat_drug$drug_name == "pegfilgrastim (peg g-csf)")] <- "pegfilgrastim"                
    pat_drug$drug_name[which(pat_drug$drug_name == "filgrastim (g-csf)")] <- "filgrastim"
    pat_drug$drug_name[which(pat_drug$drug_name == "mayo 425-20")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "oxaliplatinum")] <- "oxaliplatin"
    pat_drug$drug_name[which(pat_drug$drug_name == "5 fluorouracilum")] <- "fluorouracil"
    pat_drug$drug_name[which(pat_drug$drug_name == "irinotecan ")] <- "irinotecan"
    pat_drug$drug_name[which(pat_drug$drug_name == "c1 folfiri")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "zaltrap")] <- "aflibercept"
    pat_drug$drug_name[which(pat_drug$drug_name == "calcium foliatum")] <- "calcium acetate"
    pat_drug$drug_name[which(pat_drug$drug_name == "fluorouracilum")] <- "fluorouracil"
    pat_drug$drug_name[which(pat_drug$drug_name == "dexamethassone")] <- "dexamethasone"
    pat_drug$drug_name[which(pat_drug$drug_name == " 5-flurouracil  ")] <- "fluorouracil"
    pat_drug$drug_name[which(pat_drug$drug_name == "temodar")] <- "temozolomide"
    pat_drug$drug_name[which(pat_drug$drug_name == "bcnu")] <- "carmustine"
    pat_drug$drug_name[which(pat_drug$drug_name == "gleevec")] <- "imatinib"    
    pat_drug$drug_name[which(pat_drug$drug_name == "temozolamide")] <- "temozolomide"
    pat_drug$drug_name[which(pat_drug$drug_name == "cdr0000654697")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "at 101")] <- "isosorbide"
    pat_drug$drug_name[which(pat_drug$drug_name == "bcnu (carmustine)")] <- "carmustine"
    pat_drug$drug_name[which(pat_drug$drug_name == "pcv")] <- "penciclovir"
    pat_drug$drug_name[which(pat_drug$drug_name == "carmustine bcnu")] <- "carmustine"       
    pat_drug$drug_name[which(pat_drug$drug_name == "9 aminocamptothecin")] <- "9-aminocamptothecin"
    pat_drug$drug_name[which(pat_drug$drug_name == "bromo deoxyuridine")] <- "broxuridine"
    pat_drug$drug_name[which(pat_drug$drug_name == "sorafenib (bay 43-9006)")] <- "sorafenib"
    pat_drug$drug_name[which(pat_drug$drug_name == "gliadel wafer carmustine")] <- "carmustine"
    pat_drug$drug_name[which(pat_drug$drug_name == "ps-341 bortrezomib")] <- "bortezomib"
    pat_drug$drug_name[which(pat_drug$drug_name == "isotrectinoin (acccutane)")] <- "isotretinoin"
    pat_drug$drug_name[which(pat_drug$drug_name == "celebrex")] <- "celecoxib"
    pat_drug$drug_name[which(pat_drug$drug_name == "lomustin")] <- "lomustine"
    pat_drug$drug_name[which(pat_drug$drug_name == "eloxatin")] <- "oxaliplatin"
    pat_drug$drug_name[which(pat_drug$drug_name == "gemcitabine injection")] <- "gemcitabine"
    pat_drug$drug_name[which(pat_drug$drug_name == "irinotecan hydrochloride")] <- "irinotecan"
    pat_drug$drug_name[which(pat_drug$drug_name == "gemcitabine hcl")] <- "gemcitabine"
    pat_drug$drug_name[which(pat_drug$drug_name == "fu7")] <- "fusafungine"
    pat_drug$drug_name[which(pat_drug$drug_name == "gemcitibine")] <- "gemcitabine"
    pat_drug$drug_name[which(pat_drug$drug_name == "interferon alpha")] <- "interferon alfa"
    pat_drug$drug_name[which(pat_drug$drug_name == "temodal")] <- "temozolomide"
    pat_drug$drug_name[which(pat_drug$drug_name == "il-2")] <- "human interleukin-2"       
    pat_drug$drug_name[which(pat_drug$drug_name == "interferon")] <- "interferon alfa"
    pat_drug$drug_name[which(pat_drug$drug_name == "mustophoran")] <- "fotemustine"
    pat_drug$drug_name[which(pat_drug$drug_name == "vincristine (sulfate)")] <- "vincristine"
    pat_drug$drug_name[which(pat_drug$drug_name == "ceenu")] <- "lomustine"
    pat_drug$drug_name[which(pat_drug$drug_name == "carboplatin 5 auc")] <- "carboplatin"    
    pat_drug$drug_name[which(pat_drug$drug_name == "i 131 81c6")] <- "iodohippurate sodium i-131"
    pat_drug$drug_name[which(pat_drug$drug_name == "magi131-81c6")] <- "iodohippurate sodium i-131"
    pat_drug$drug_name[which(pat_drug$drug_name == "i131-81c6")] <- "iodohippurate sodium i-131"
    pat_drug$drug_name[which(pat_drug$drug_name == "mr1-1")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "karenitecin")] <- "cositecan"
    pat_drug$drug_name[which(pat_drug$drug_name == "benzylguanine")] <- "6-o-benzylguanine"
    pat_drug$drug_name[which(pat_drug$drug_name == "themozolomide")] <- "temozolomide"
    pat_drug$drug_name[which(pat_drug$drug_name == "carmustin")] <- "carmustine"                                   
    pat_drug$drug_name[which(pat_drug$drug_name == "premetrexed")] <- "pemetrexed"
    pat_drug$drug_name[which(pat_drug$drug_name == "vepesid")] <- "etoposide"    
	pat_drug$drug_name[which(pat_drug$drug_name == "vinorelbin tartrate")] <- "vinorelbine"
	pat_drug$drug_name[which(pat_drug$drug_name == "vinorelbine tartrate")] <- "vinorelbine"
    pat_drug$drug_name[which(pat_drug$drug_name == "tl32711")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "docetaxelum")] <- "docetaxel"
    pat_drug$drug_name[which(pat_drug$drug_name == "plfe")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "5-fluorouracilum")] <- "fluorouracil"
    pat_drug$drug_name[which(pat_drug$drug_name == "flc")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "cepecitabine")] <- "capecitabine"
    pat_drug$drug_name[which(pat_drug$drug_name == "copolang")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "doxifluridin")] <- "doxifluridine"
    pat_drug$drug_name[which(pat_drug$drug_name == "copolang caps")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "micomycin")] <- "ciclopirox"
    pat_drug$drug_name[which(pat_drug$drug_name == "ts-1")] <- "tegafur"
    pat_drug$drug_name[which(pat_drug$drug_name == "cddp")] <- "cisplatin"
    pat_drug$drug_name[which(pat_drug$drug_name == "xeloda(capecitabine)")] <- "capecitabine"
    pat_drug$drug_name[which(pat_drug$drug_name == "eloxatin(oxaliplatin)")] <- "oxaliplatin"
    pat_drug$drug_name[which(pat_drug$drug_name == "unkown")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "taxel")] <- "paclitaxel"
    pat_drug$drug_name[which(pat_drug$drug_name == "oxaloplatin")] <- "oxaliplatin"
    pat_drug$drug_name[which(pat_drug$drug_name == "xelox")] <- "capecitabine"             
    pat_drug$drug_name[which(pat_drug$drug_name == "etoposidium")] <- "etoposide"
    pat_drug$drug_name[which(pat_drug$drug_name == "flo")] <- ""   
    pat_drug$drug_name[which(pat_drug$drug_name == "dexamethasone phosphate")] <- "dexamethasone"
    pat_drug$drug_name[which(pat_drug$drug_name == "capecytabinum")] <- "capecitabine"
    pat_drug$drug_name[which(pat_drug$drug_name == "epirubicinum")] <- "epirubicin"        
    pat_drug$drug_name[which(pat_drug$drug_name == "cisplatinum")] <- "cisplatin"
    pat_drug$drug_name[which(pat_drug$drug_name == "leukine")] <- "sargramostim"
    pat_drug$drug_name[which(pat_drug$drug_name == "medrol")] <- "methylprednisolone"
    pat_drug$drug_name[which(pat_drug$drug_name == "halaven")] <- "eribulin"
    pat_drug$drug_name[which(pat_drug$drug_name == "polyplatillen")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "etoposide phosphate")] <- "etoposide"
    pat_drug$drug_name[which(pat_drug$drug_name == "clinical trial")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "paraplatin")] <- "carboplatin"       
    pat_drug$drug_name[which(pat_drug$drug_name == "reolysin")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "mk2206")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "azd6244")] <- "selumetinib" 
    pat_drug$drug_name[which(pat_drug$drug_name == "ccnu")] <- "lomustine"        
    pat_drug$drug_name[which(pat_drug$drug_name == "carboplatinum")] <- "carboplatin"
    pat_drug$drug_name[which(pat_drug$drug_name == "mithramycin")] <- "plicamycin"
    pat_drug$drug_name[which(pat_drug$drug_name == "gemzar (gemcitabine)")] <- "gemcitabine"        
    pat_drug$drug_name[which(pat_drug$drug_name == "etoposide (vp-16)")] <- "etoposide"
    pat_drug$drug_name[which(pat_drug$drug_name == "tarveca")] <- "erlotinib"
    pat_drug$drug_name[which(pat_drug$drug_name == "alimta")] <- "pemetrexed"
    pat_drug$drug_name[which(pat_drug$drug_name == "iressa")] <- "gefitinib"
    pat_drug$drug_name[which(pat_drug$drug_name == "iressa placebo vs. drug")] <- "gefitinib"
    pat_drug$drug_name[which(pat_drug$drug_name == "cav trial")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "ironotecan")] <- "irinotecan"
    pat_drug$drug_name[which(pat_drug$drug_name == "placebo")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "vinorelbin")] <- "vinorelbine"
    pat_drug$drug_name[which(pat_drug$drug_name == "not specified")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "mdx-1106 clinical trial")] <- "nivolumab"
    pat_drug$drug_name[which(pat_drug$drug_name == "tarceva ")] <- "erlotinib"
    pat_drug$drug_name[which(pat_drug$drug_name == "pemetrexed disodium")] <- "pemetrexed"
    pat_drug$drug_name[which(pat_drug$drug_name == "almita")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "erlotonib")] <- "erlotinib"
    pat_drug$drug_name[which(pat_drug$drug_name == "b12")] <- "cyanocobalamin"                  
    pat_drug$drug_name[which(pat_drug$drug_name == "lomustine (ccnu)")] <- "lomustine"
    pat_drug$drug_name[which(pat_drug$drug_name == "cilengtide")] <- "cilengitide"
    pat_drug$drug_name[which(pat_drug$drug_name == "temozlomide")] <- "temozolomide"
    pat_drug$drug_name[which(pat_drug$drug_name == "irintotecan")] <- "irinotecan"
    pat_drug$drug_name[which(pat_drug$drug_name == "tomaxifen")] <- "tamoxifen"
    pat_drug$drug_name[which(pat_drug$drug_name == "tarceva (erlotinib)")] <- "erlotinib"
    pat_drug$drug_name[which(pat_drug$drug_name == "doxetaxol")] <- "doxycycline"
    pat_drug$drug_name[which(pat_drug$drug_name == "gliadel")] <- "carmustine"
    pat_drug$drug_name[which(pat_drug$drug_name == "tamoxiten")] <- "tamoxifen"
    pat_drug$drug_name[which(pat_drug$drug_name == "temazolomide")] <- "temozolomide"    
    pat_drug$drug_name[which(pat_drug$drug_name == "gliadel wafers")] <- "carmustine"
    pat_drug$drug_name[which(pat_drug$drug_name == "cloretazine")] <- "laromustine"    
    pat_drug$drug_name[which(pat_drug$drug_name == "karerltecin")] <- "cositecan"    
    pat_drug$drug_name[which(pat_drug$drug_name == "su101")] <- "leflunomide"
    pat_drug$drug_name[which(pat_drug$drug_name == "onyx-015")] <- ""
    pat_drug$drug_name[which(pat_drug$drug_name == "hydroyurea")] <- "hydroxyurea"
    pat_drug$drug_name[which(pat_drug$drug_name == "carmustine (bcnu)")] <- "carmustine"
    pat_drug$drug_name[which(pat_drug$drug_name == "cis-retinoic acid")] <- "isotretinoin"        
    pat_drug$drug_name[grep("5-fluor", pat_drug$drug_name)] <- "fluorouracil"                   
    pat_drug$drug_name[grep("5-flour", pat_drug$drug_name)] <- "fluorouracil"

    file <- paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/04_mm_ko_drug_used/04_1_correc_drug/",proj_tcga_sel[i],"/pat_drug_corr_",proj_tcga_sel[i],".rds",sep="")
    saveRDS(pat_drug, file=file)    
                                
}                 